### Overview



In [1]:
import os
import importlib
import random
import librosa
import torch
import umap
import umap.plot
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import torch.nn.functional as F
import torch

import numpy as np
from TTS.utils.generic_utils import load_config
from tqdm import tqdm
from TTS.utils.speakers import save_speaker_mapping, load_speaker_mapping

# you may need to change this depending on your system
os.environ['CUDA_VISIBLE_DEVICES']='0'


colormap = np.array([
            [76, 255, 0],
            [0, 127, 70],
            [255, 0, 0],
            [255, 217, 38],
            [0, 135, 255],
            [165, 0, 165],
            [255, 167, 255],
            [0, 255, 255],
            [255, 96, 38],
            [142, 76, 0],
            [33, 0, 127],
            [0, 0, 0],
            [183, 183, 183],
                ]) / 255


In [2]:
## Teste using Prototypical-ResNetSE34L (trained on VoxCeleb) for BRspeech and VCTK

In [3]:
# Set constants
ROOT_PATH = '../../' 
SPEAKER_JSON_PATH = os.path.join(ROOT_PATH, 'speakers-pt-BRSpeech-beta3-Angular Prototypical-ResNetSE34L.json')

#test load speaker json
speaker_mapping = load_speaker_mapping(SPEAKER_JSON_PATH)
if speaker_mapping == {}:
    raise ValueError("File Not Found Error, Verify SPEAKER_JSON_PATH")
    
#config plot
max_speakers = 9999
samples_per_speaker = 1000


speakers_samples = {}
embeds = []
labels = []
colors_id = []
num_speakers = 0
keys_mapping = list(speaker_mapping.keys())
random.shuffle(keys_mapping)
for i, sample in enumerate(keys_mapping):
    if speaker_mapping[sample]['name'] in speakers_samples.keys():
        if speakers_samples[speaker_mapping[sample]['name']]['n_samples'] > samples_per_speaker:
            continue
        speakers_samples[speaker_mapping[sample]['name']]['n_samples'] = speakers_samples[speaker_mapping[sample]['name']]['n_samples'] + 1
        aux =  speakers_samples[speaker_mapping[sample]['name']]['samples']
        aux.append(speaker_mapping[sample]['embedding'])
        speakers_samples[speaker_mapping[sample]['name']]['samples'] = aux
    else:
        if num_speakers >= max_speakers: #max speakers
            continue
        speakers_samples[speaker_mapping[sample]['name']] = {'n_samples':1,'samples':[speaker_mapping[sample]['embedding']]}    
        num_speakers += 1
        

speakers_samples_keys = list(speakers_samples.keys())
print(" Num speakers: ", len(speakers_samples_keys))
for i, sample in enumerate(speakers_samples_keys):
    speaker_embeddings =  speakers_samples[sample]['samples']
    speakers_samples[sample]['centroid'] = np.mean(np.array(speaker_embeddings), axis=0)

total_incorect = 0
total_testes = 0
inte = 0
for i, sample in enumerate(speakers_samples_keys): # for em cada speaker
    inte = 0
    speakers_samples[sample]['n_samples_invade_space'] = 0
    for emb in speakers_samples[sample]['samples']: # for nos embeddings do speaker
        #calcular dist emb e centroid
        dist_self = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(speakers_samples[sample]['centroid'])).detach().cpu().numpy();
        
        for j in range(len(speakers_samples_keys)): # for em cada 1 dos outros speakers
            if j == i:
                continue
            centroid = speakers_samples[speakers_samples_keys[j]]['centroid']
            #calcula a dist entre o emb com o centroid dos outros locutores    
            dist_other = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(centroid)).detach().cpu().numpy()
            #print(dist_other, np.array(emb).shape,np.array(centroid).shape )    
            if dist_self > dist_other:
                    speakers_samples[sample]['n_samples_invade_space'] += 1
            inte+=1
    print("Total de comparações para o Speaker, ",sample.replace('\n',''), ' é de ', inte)
    print('Number of sample incorrect:',speakers_samples[sample]['n_samples_invade_space'])
    total_testes+=inte
    total_incorect+=speakers_samples[sample]['n_samples_invade_space']

    
print("De ",total_testes, " Errou ", total_incorect, "erro porcento: ",total_incorect*100/total_testes)  

 Num speakers:  44
Total de comparações para o Speaker,  SL  é de  43043
Number of sample incorrect: 132
Total de comparações para o Speaker,  ED  é de  43043
Number of sample incorrect: 1629
Total de comparações para o Speaker,  MR  é de  21457
Number of sample incorrect: 319
Total de comparações para o Speaker,  GO  é de  43043
Number of sample incorrect: 1146
Total de comparações para o Speaker,  PE  é de  43043
Number of sample incorrect: 662
Total de comparações para o Speaker,  AS  é de  43043
Number of sample incorrect: 1135
Total de comparações para o Speaker,  JC  é de  43043
Number of sample incorrect: 10999
Total de comparações para o Speaker,  RL  é de  17587
Number of sample incorrect: 292
Total de comparações para o Speaker,  LN  é de  43043
Number of sample incorrect: 1027
Total de comparações para o Speaker,  TH  é de  43043
Number of sample incorrect: 308
Total de comparações para o Speaker,  CZ  é de  7396
Number of sample incorrect: 28
Total de comparações para o Spe

In [4]:
# Set constants
ROOT_PATH = '../../'
SPEAKER_JSON_PATH = os.path.join(ROOT_PATH, 'speakers-en-vctk-Angular Prototypical-ResNetSE34L.json')

#test load speaker json
speaker_mapping = load_speaker_mapping(SPEAKER_JSON_PATH)
if speaker_mapping == {}:
    raise ValueError("File Not Found Error, Verify SPEAKER_JSON_PATH")
    
#config plot
max_speakers = 9999
samples_per_speaker = 1000


speakers_samples = {}
embeds = []
labels = []
colors_id = []
num_speakers = 0
keys_mapping = list(speaker_mapping.keys())
random.shuffle(keys_mapping)
for i, sample in enumerate(keys_mapping):
    if speaker_mapping[sample]['name'] in speakers_samples.keys():
        if speakers_samples[speaker_mapping[sample]['name']]['n_samples'] > samples_per_speaker:
            continue
        speakers_samples[speaker_mapping[sample]['name']]['n_samples'] = speakers_samples[speaker_mapping[sample]['name']]['n_samples'] + 1
        aux =  speakers_samples[speaker_mapping[sample]['name']]['samples']
        aux.append(speaker_mapping[sample]['embedding'])
        speakers_samples[speaker_mapping[sample]['name']]['samples'] = aux
    else:
        if num_speakers >= max_speakers: #max speakers
            continue
        speakers_samples[speaker_mapping[sample]['name']] = {'n_samples':1,'samples':[speaker_mapping[sample]['embedding']]}    
        num_speakers += 1
        

speakers_samples_keys = list(speakers_samples.keys())
print(" Num speakers: ", len(speakers_samples_keys))
for i, sample in enumerate(speakers_samples_keys):
    speaker_embeddings =  speakers_samples[sample]['samples']
    speakers_samples[sample]['centroid'] = np.mean(np.array(speaker_embeddings), axis=0)

total_incorect = 0
total_testes = 0
inte = 0
for i, sample in enumerate(speakers_samples_keys): # for em cada speaker
    inte = 0
    speakers_samples[sample]['n_samples_invade_space'] = 0
    for emb in speakers_samples[sample]['samples']: # for nos embeddings do speaker
        #calcular dist emb e centroid
        dist_self = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(speakers_samples[sample]['centroid'])).detach().cpu().numpy();
        
        for j in range(len(speakers_samples_keys)): # for em cada 1 dos outros speakers
            if j == i:
                continue
            centroid = speakers_samples[speakers_samples_keys[j]]['centroid']
            #calcula a dist entre o emb com o centroid dos outros locutores    
            dist_other = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(centroid)).detach().cpu().numpy()
            #print(dist_other, np.array(emb).shape,np.array(centroid).shape )    
            if dist_self > dist_other:
                    speakers_samples[sample]['n_samples_invade_space'] += 1
            inte+=1
    print("Total de comparações para o Speaker, ",sample.replace('\n',''), ' é de ', inte)
    print('Number of sample incorrect:',speakers_samples[sample]['n_samples_invade_space'])
    total_testes+=inte
    total_incorect+=speakers_samples[sample]['n_samples_invade_space']

    
print("De ",total_testes, " Errou ", total_incorect, "erro porcento: ",total_incorect*100/total_testes)   

 Num speakers:  108
Total de comparações para o Speaker,  p250  é de  51467
Number of sample incorrect: 4571
Total de comparações para o Speaker,  p258  é de  44298
Number of sample incorrect: 4305
Total de comparações para o Speaker,  p226  é de  38092
Number of sample incorrect: 3744
Total de comparações para o Speaker,  p313  é de  45047
Number of sample incorrect: 5533
Total de comparações para o Speaker,  p292  é de  43977
Number of sample incorrect: 3431
Total de comparações para o Speaker,  p263  é de  50397
Number of sample incorrect: 1782
Total de comparações para o Speaker,  p298  é de  43335
Number of sample incorrect: 5259
Total de comparações para o Speaker,  p360  é de  45368
Number of sample incorrect: 4392
Total de comparações para o Speaker,  p229  é de  40553
Number of sample incorrect: 3367
Total de comparações para o Speaker,  p262  é de  42158
Number of sample incorrect: 3168
Total de comparações para o Speaker,  p307  é de  44940
Number of sample incorrect: 925
To

Total de comparações para o Speaker,  p254  é de  42479
Number of sample incorrect: 3008
Total de comparações para o Speaker,  p288  é de  44084
Number of sample incorrect: 3629
Total de comparações para o Speaker,  p317  é de  45261
Number of sample incorrect: 2314
Total de comparações para o Speaker,  p275  é de  44512
Number of sample incorrect: 4695
Total de comparações para o Speaker,  p376  é de  31244
Number of sample incorrect: 5761
Total de comparações para o Speaker,  p237  é de  36273
Number of sample incorrect: 3358
Total de comparações para o Speaker,  p248  é de  40232
Number of sample incorrect: 2445
Total de comparações para o Speaker,  p270  é de  49434
Number of sample incorrect: 3079
Total de comparações para o Speaker,  p302  é de  33277
Number of sample incorrect: 5213
Total de comparações para o Speaker,  p308  é de  45261
Number of sample incorrect: 3548
Total de comparações para o Speaker,  p304  é de  45261
Number of sample incorrect: 4207
Total de comparações 

In [5]:
## Teste using Speech2Phone (official implementation) for BRspeech and VCTK

In [6]:

# Set constants
ROOT_PATH = '../../'
SPEAKER_JSON_PATH = os.path.join(ROOT_PATH, 'speakers-pt-Speech2Phone-BRSpeech-beta3.json')


#test load speaker json
speaker_mapping = load_speaker_mapping(SPEAKER_JSON_PATH)
if speaker_mapping == {}:
    raise ValueError("File Not Found Error, Verify SPEAKER_JSON_PATH")
    
#config plot
max_speakers = 9999
samples_per_speaker = 1000


speakers_samples = {}
embeds = []
labels = []
colors_id = []
num_speakers = 0
keys_mapping = list(speaker_mapping.keys())
random.shuffle(keys_mapping)
for i, sample in enumerate(keys_mapping):
    if speaker_mapping[sample]['name'] in speakers_samples.keys():
        if speakers_samples[speaker_mapping[sample]['name']]['n_samples'] > samples_per_speaker:
            continue
        speakers_samples[speaker_mapping[sample]['name']]['n_samples'] = speakers_samples[speaker_mapping[sample]['name']]['n_samples'] + 1
        aux =  speakers_samples[speaker_mapping[sample]['name']]['samples']
        aux.append(speaker_mapping[sample]['embedding'])
        speakers_samples[speaker_mapping[sample]['name']]['samples'] = aux
    else:
        if num_speakers >= max_speakers: #max speakers
            continue
        speakers_samples[speaker_mapping[sample]['name']] = {'n_samples':1,'samples':[speaker_mapping[sample]['embedding']]}    
        num_speakers += 1
        

speakers_samples_keys = list(speakers_samples.keys())
print(" Num speakers: ", len(speakers_samples_keys))
for i, sample in enumerate(speakers_samples_keys):
    speaker_embeddings =  speakers_samples[sample]['samples']
    speakers_samples[sample]['centroid'] = np.mean(np.array(speaker_embeddings), axis=0)

total_incorect = 0
total_testes = 0
inte = 0
for i, sample in enumerate(speakers_samples_keys): # for em cada speaker
    inte = 0
    speakers_samples[sample]['n_samples_invade_space'] = 0
    for emb in speakers_samples[sample]['samples']: # for nos embeddings do speaker
        #calcular dist emb e centroid
        dist_self = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(speakers_samples[sample]['centroid'])).detach().cpu().numpy();
        
        for j in range(len(speakers_samples_keys)): # for em cada 1 dos outros speakers
            if j == i:
                continue
            centroid = speakers_samples[speakers_samples_keys[j]]['centroid']
            #calcula a dist entre o emb com o centroid dos outros locutores    
            dist_other = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(centroid)).detach().cpu().numpy()
            #print(dist_other, np.array(emb).shape,np.array(centroid).shape )    
            if dist_self > dist_other:
                    speakers_samples[sample]['n_samples_invade_space'] += 1
            inte+=1
    print("Total de comparações para o Speaker, ",sample.replace('\n',''), ' é de ', inte)
    print('Number of sample incorrect:',speakers_samples[sample]['n_samples_invade_space'])
    total_testes+=inte
    total_incorect+=speakers_samples[sample]['n_samples_invade_space']

    
print("De ",total_testes, " Errou ", total_incorect, "erro porcento: ",total_incorect*100/total_testes)      

 Num speakers:  44
Total de comparações para o Speaker,  LR  é de  26617
Number of sample incorrect: 396
Total de comparações para o Speaker,  AS  é de  43043
Number of sample incorrect: 3462
Total de comparações para o Speaker,  ED  é de  43043
Number of sample incorrect: 343
Total de comparações para o Speaker,  LN  é de  43043
Number of sample incorrect: 2789
Total de comparações para o Speaker,  MM  é de  43043
Number of sample incorrect: 55
Total de comparações para o Speaker,  JA  é de  43043
Number of sample incorrect: 358
Total de comparações para o Speaker,  GO  é de  43043
Number of sample incorrect: 574
Total de comparações para o Speaker,  GM  é de  43043
Number of sample incorrect: 223
Total de comparações para o Speaker,  SL  é de  43043
Number of sample incorrect: 352
Total de comparações para o Speaker,  TH  é de  43043
Number of sample incorrect: 743
Total de comparações para o Speaker,  JP  é de  5934
Number of sample incorrect: 0
Total de comparações para o Speaker, 

In [7]:

# Set constants
ROOT_PATH = '../../'
SPEAKER_JSON_PATH = os.path.join(ROOT_PATH, 'speakers-Speech2Phone-vctk.json')


#test load speaker json
speaker_mapping = load_speaker_mapping(SPEAKER_JSON_PATH)
if speaker_mapping == {}:
    raise ValueError("File Not Found Error, Verify SPEAKER_JSON_PATH")
    
#config plot
max_speakers = 9999
samples_per_speaker = 1000


speakers_samples = {}
embeds = []
labels = []
colors_id = []
num_speakers = 0
keys_mapping = list(speaker_mapping.keys())
random.shuffle(keys_mapping)
for i, sample in enumerate(keys_mapping):
    if speaker_mapping[sample]['name'] in speakers_samples.keys():
        if speakers_samples[speaker_mapping[sample]['name']]['n_samples'] > samples_per_speaker:
            continue
        speakers_samples[speaker_mapping[sample]['name']]['n_samples'] = speakers_samples[speaker_mapping[sample]['name']]['n_samples'] + 1
        aux =  speakers_samples[speaker_mapping[sample]['name']]['samples']
        aux.append(speaker_mapping[sample]['embedding'])
        speakers_samples[speaker_mapping[sample]['name']]['samples'] = aux
    else:
        if num_speakers >= max_speakers: #max speakers
            continue
        speakers_samples[speaker_mapping[sample]['name']] = {'n_samples':1,'samples':[speaker_mapping[sample]['embedding']]}    
        num_speakers += 1
        

speakers_samples_keys = list(speakers_samples.keys())
print(" Num speakers: ", len(speakers_samples_keys))
for i, sample in enumerate(speakers_samples_keys):
    speaker_embeddings =  speakers_samples[sample]['samples']
    speakers_samples[sample]['centroid'] = np.mean(np.array(speaker_embeddings), axis=0)

total_incorect = 0
total_testes = 0
inte = 0
for i, sample in enumerate(speakers_samples_keys): # for em cada speaker
    inte = 0
    speakers_samples[sample]['n_samples_invade_space'] = 0
    for emb in speakers_samples[sample]['samples']: # for nos embeddings do speaker
        #calcular dist emb e centroid
        dist_self = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(speakers_samples[sample]['centroid'])).detach().cpu().numpy();
        
        for j in range(len(speakers_samples_keys)): # for em cada 1 dos outros speakers
            if j == i:
                continue
            centroid = speakers_samples[speakers_samples_keys[j]]['centroid']
            #calcula a dist entre o emb com o centroid dos outros locutores    
            dist_other = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(centroid)).detach().cpu().numpy()
            #print(dist_other, np.array(emb).shape,np.array(centroid).shape )    
            if dist_self > dist_other:
                    speakers_samples[sample]['n_samples_invade_space'] += 1
            inte+=1
    print("Total de comparações para o Speaker, ",sample.replace('\n',''), ' é de ', inte)
    print('Number of sample incorrect:',speakers_samples[sample]['n_samples_invade_space'])
    total_testes+=inte
    total_incorect+=speakers_samples[sample]['n_samples_invade_space']

    
print("De ",total_testes, " Errou ", total_incorect, "erro porcento: ",total_incorect*100/total_testes)  

 Num speakers:  44
Total de comparações para o Speaker,  AS  é de  43043
Number of sample incorrect: 3434
Total de comparações para o Speaker,  GO  é de  43043
Number of sample incorrect: 540
Total de comparações para o Speaker,  JC  é de  43043
Number of sample incorrect: 354
Total de comparações para o Speaker,  GM  é de  43043
Number of sample incorrect: 230
Total de comparações para o Speaker,  TH  é de  43043
Number of sample incorrect: 756
Total de comparações para o Speaker,  GT  é de  10191
Number of sample incorrect: 2
Total de comparações para o Speaker,  LN  é de  43043
Number of sample incorrect: 3006
Total de comparações para o Speaker,  ED  é de  43043
Number of sample incorrect: 287
Total de comparações para o Speaker,  JA  é de  43043
Number of sample incorrect: 344
Total de comparações para o Speaker,  VC  é de  43043
Number of sample incorrect: 982
Total de comparações para o Speaker,  RL  é de  14620
Number of sample incorrect: 15
Total de comparações para o Speaker,

In [8]:
## Teste using GE2E mozilla implementation (trained on LibriTTS) for BRspeech and VCTK

In [9]:

# Set constants
ROOT_PATH = '../../'
SPEAKER_JSON_PATH = os.path.join(ROOT_PATH, 'speakers-vctk-en.json')


#test load speaker json
speaker_mapping = load_speaker_mapping(SPEAKER_JSON_PATH)
if speaker_mapping == {}:
    raise ValueError("File Not Found Error, Verify SPEAKER_JSON_PATH")
    
#config plot
max_speakers = 9999
samples_per_speaker = 1000


speakers_samples = {}
embeds = []
labels = []
colors_id = []
num_speakers = 0
keys_mapping = list(speaker_mapping.keys())
random.shuffle(keys_mapping)
for i, sample in enumerate(keys_mapping):
    if speaker_mapping[sample]['name'] in speakers_samples.keys():
        if speakers_samples[speaker_mapping[sample]['name']]['n_samples'] > samples_per_speaker:
            continue
        speakers_samples[speaker_mapping[sample]['name']]['n_samples'] = speakers_samples[speaker_mapping[sample]['name']]['n_samples'] + 1
        aux =  speakers_samples[speaker_mapping[sample]['name']]['samples']
        aux.append(speaker_mapping[sample]['embedding'])
        speakers_samples[speaker_mapping[sample]['name']]['samples'] = aux
    else:
        if num_speakers >= max_speakers: #max speakers
            continue
        speakers_samples[speaker_mapping[sample]['name']] = {'n_samples':1,'samples':[speaker_mapping[sample]['embedding']]}    
        num_speakers += 1
        

speakers_samples_keys = list(speakers_samples.keys())
print(" Num speakers: ", len(speakers_samples_keys))
for i, sample in enumerate(speakers_samples_keys):
    speaker_embeddings =  speakers_samples[sample]['samples']
    speakers_samples[sample]['centroid'] = np.mean(np.array(speaker_embeddings), axis=0)

total_incorect = 0
total_testes = 0
inte = 0
for i, sample in enumerate(speakers_samples_keys): # for em cada speaker
    inte = 0
    speakers_samples[sample]['n_samples_invade_space'] = 0
    for emb in speakers_samples[sample]['samples']: # for nos embeddings do speaker
        #calcular dist emb e centroid
        dist_self = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(speakers_samples[sample]['centroid'])).detach().cpu().numpy();
        
        for j in range(len(speakers_samples_keys)): # for em cada 1 dos outros speakers
            if j == i:
                continue
            centroid = speakers_samples[speakers_samples_keys[j]]['centroid']
            #calcula a dist entre o emb com o centroid dos outros locutores    
            dist_other = F.pairwise_distance(torch.FloatTensor([emb]),torch.FloatTensor(centroid)).detach().cpu().numpy()
            #print(dist_other, np.array(emb).shape,np.array(centroid).shape )    
            if dist_self > dist_other:
                    speakers_samples[sample]['n_samples_invade_space'] += 1
            inte+=1
    print("Total de comparações para o Speaker, ",sample.replace('\n',''), ' é de ', inte)
    print('Number of sample incorrect:',speakers_samples[sample]['n_samples_invade_space'])
    total_testes+=inte
    total_incorect+=speakers_samples[sample]['n_samples_invade_space']

    
print("De ",total_testes, " Errou ", total_incorect, "erro porcento: ",total_incorect*100/total_testes)  

 Num speakers:  108
Total de comparações para o Speaker,  p246  é de  38306
Number of sample incorrect: 536
Total de comparações para o Speaker,  p316  é de  45047
Number of sample incorrect: 35
Total de comparações para o Speaker,  p277  é de  49113
Number of sample incorrect: 1215
Total de comparações para o Speaker,  p279  é de  43335
Number of sample incorrect: 612
Total de comparações para o Speaker,  p267  é de  44619
Number of sample incorrect: 571
Total de comparações para o Speaker,  p252  é de  42265
Number of sample incorrect: 184
Total de comparações para o Speaker,  p232  é de  44084
Number of sample incorrect: 270
Total de comparações para o Speaker,  p274  é de  49862
Number of sample incorrect: 642
Total de comparações para o Speaker,  p340  é de  45261
Number of sample incorrect: 563
Total de comparações para o Speaker,  p238  é de  48578
Number of sample incorrect: 792
Total de comparações para o Speaker,  p307  é de  44940
Number of sample incorrect: 256
Total de com

Total de comparações para o Speaker,  p302  é de  33277
Number of sample incorrect: 135
Total de comparações para o Speaker,  p283  é de  50183
Number of sample incorrect: 1277
Total de comparações para o Speaker,  p234  é de  38199
Number of sample incorrect: 511
Total de comparações para o Speaker,  p299  é de  43335
Number of sample incorrect: 332
Total de comparações para o Speaker,  p227  é de  41623
Number of sample incorrect: 337
Total de comparações para o Speaker,  p285  é de  43121
Number of sample incorrect: 388
Total de comparações para o Speaker,  p306  é de  37557
Number of sample incorrect: 742
Total de comparações para o Speaker,  p280  é de  43870
Number of sample incorrect: 1185
Total de comparações para o Speaker,  p257  é de  46438
Number of sample incorrect: 1954
Total de comparações para o Speaker,  p240  é de  40339
Number of sample incorrect: 855
Total de comparações para o Speaker,  p226  é de  38092
Number of sample incorrect: 460
Total de comparações para o S